# Day3_1: 예외 처리와 디버깅 (Exceptions)

## 📚 학습 목표

1. **에러 핸들링**: `try`, `except`, `else`, `finally` 완전 정복
2. **에러 체이닝**: `raise from`, `ExceptionGroup` (Python 3.11+)
3. **로깅(Logging)**: `print` 대신 `logging` 모듈 사용하기
4. **실무 활용**: 크롤링, 배치 작업에서 안전한 에러 처리

---

## 🎯 왜 중요한가요?

**분석가 관점**:
- 데이터 크롤링 중 일부 페이지 실패 → 스크립트 중단 방지
- 엑셀 파일 읽기 실패 → 다른 파일 계속 처리
- API 호출 타임아웃 → 재시도 로직 구현
- 배치 작업 로그 → 어디서 실패했는지 추적

**안전성 비교**:
```python
# 위험한 코드 (에러 시 전체 중단)
for url in urls:
    data = fetch(url)  # 하나라도 실패하면 멈춤
    results.append(data)

# 안전한 코드 (에러 처리)
for url in urls:
    try:
        data = fetch(url)
        results.append(data)
    except Exception as e:
        logging.error(f"{url} 실패: {e}")
        continue  # 다음 URL 계속 처리
```

→ **스크립트가 죽지 않고 끝까지 실행!**

---

## 📖 핵심 개념

### 1. 기본 예외 처리

**try-except 구조**:
```python
try:
    # 시도할 코드
    risky_operation()
except SpecificError:
    # 특정 에러 처리
    handle_error()
except Exception as e:
    # 모든 에러 처리
    print(f"에러: {e}")
else:
    # 에러 없을 때만 실행
    success_action()
finally:
    # 항상 실행 (정리 작업)
    cleanup()
```

### 2. 주요 예외 타입

```python
# 파일 관련
FileNotFoundError  # 파일이 없을 때
PermissionError    # 권한 없을 때

# 데이터 관련
ValueError         # 잘못된 값
TypeError          # 잘못된 타입
KeyError           # 딕셔너리 키 없음
IndexError         # 리스트 인덱스 범위 초과

# 네트워크 관련
TimeoutError       # 시간 초과
ConnectionError    # 연결 실패

# 사용자 정의
class CustomError(Exception):
    pass
```

### 3. 예외 체이닝 (raise from)

```python
try:
    low_level_operation()
except LowLevelError as e:
    # 원인을 보존하며 새 에러 발생
    raise HighLevelError("작업 실패") from e
```

### 4. 로깅 모듈

```python
import logging

# 기본 설정
logging.basicConfig(level=logging.INFO)

# 로그 레벨
logging.debug("디버그 정보")      # 개발 시
logging.info("일반 정보")         # 진행 상황
logging.warning("경고")           # 주의 필요
logging.error("에러")             # 에러 발생
logging.critical("치명적 에러")   # 심각한 문제
```

### 5. 실무 패턴

**재시도 로직**:
```python
def retry(func, max_attempts=3):
    for attempt in range(max_attempts):
        try:
            return func()
        except Exception as e:
            if attempt == max_attempts - 1:
                raise
            logging.warning(f"재시도 {attempt+1}/{max_attempts}")
```

**안전한 리소스 관리**:
```python
# with 문 사용 (권장)
with open('file.txt') as f:
    data = f.read()  # 자동으로 파일 닫힘

# try-finally 사용
f = open('file.txt')
try:
    data = f.read()
finally:
    f.close()  # 반드시 실행
```

---

## 🎯 실습 퀴즈

**난이도 표시**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 기본 try-except ⭐

**문제**: 사용자 입력을 정수로 변환하는 안전한 함수를 작성하세요.

**요구사항**:
- 정수로 변환 가능하면 정수 반환
- 변환 불가능하면 `None` 반환
- `ValueError` 예외 처리

**함수 시그니처**:
```python
def safe_int(value):
    """
    안전하게 정수로 변환
    
    Args:
        value: 변환할 값
    
    Returns:
        int or None: 변환된 정수 또는 None
    """
    pass
```

**테스트**:
```python
print(safe_int("123"))    # 123
print(safe_int("abc"))    # None
print(safe_int("45.6"))   # None
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. 여러 예외 타입 처리 ⭐⭐

**문제**: 리스트에서 안전하게 값을 가져오는 함수를 작성하세요.

**요구사항**:
- 인덱스가 유효하면 해당 값 반환
- `IndexError`: "인덱스 범위 초과" 메시지와 함께 `None` 반환
- `TypeError`: "잘못된 타입" 메시지와 함께 `None` 반환
- 기타 예외: "알 수 없는 에러" 메시지와 함께 `None` 반환

**함수 시그니처**:
```python
def safe_get(data, index):
    """
    안전하게 리스트에서 값 가져오기
    
    Args:
        data: 리스트
        index: 인덱스
    
    Returns:
        값 또는 None (에러 메시지 출력)
    """
    pass
```

**테스트**:
```python
data = [10, 20, 30]
print(safe_get(data, 1))      # 20
print(safe_get(data, 10))     # None (인덱스 범위 초과)
print(safe_get(data, "a"))    # None (잘못된 타입)
```

In [ ]:
# 여기에 코드를 작성하세요


### Q3. try-except-else-finally 완전 정복 ⭐⭐⭐

**문제**: 파일을 읽고 처리하는 함수를 작성하세요.

**시나리오**:
파일을 읽어서 줄 수를 세고, 처리 결과를 로그로 남깁니다.

**요구사항**:
- `try`: 파일 읽기 시도
- `except FileNotFoundError`: "파일을 찾을 수 없습니다" 출력 후 0 반환
- `except PermissionError`: "파일 읽기 권한이 없습니다" 출력 후 -1 반환
- `else`: 에러 없으면 "파일 처리 완료" 출력
- `finally`: "파일 처리 종료" 항상 출력

**함수 시그니처**:
```python
def count_file_lines(filepath):
    """
    파일의 줄 수를 세기
    
    Args:
        filepath: 파일 경로
    
    Returns:
        int: 줄 수 (에러 시 0 또는 -1)
    """
    pass
```

**힌트**: `with open()` 대신 `try-finally`를 사용하여 명시적으로 처리하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 사용자 정의 예외 ⭐⭐⭐

**문제**: 데이터 검증을 위한 커스텀 예외를 만들고 사용하세요.

**시나리오**:
사용자 데이터를 검증하는 함수를 작성합니다.

**요구사항**:
1. `ValidationError` 예외 클래스 정의
2. `validate_user` 함수 작성:
   - 나이가 0 미만이거나 150 초과: `ValidationError` 발생
   - 이름이 빈 문자열: `ValidationError` 발생
   - 이메일에 '@' 없음: `ValidationError` 발생
3. 각 에러에 의미있는 메시지 포함

**클래스 및 함수 시그니처**:
```python
class ValidationError(Exception):
    """데이터 검증 실패 예외"""
    pass

def validate_user(name, age, email):
    """
    사용자 데이터 검증
    
    Args:
        name: 이름
        age: 나이
        email: 이메일
    
    Raises:
        ValidationError: 검증 실패 시
    
    Returns:
        dict: 검증된 사용자 정보
    """
    pass
```

**테스트**:
```python
# 정상
print(validate_user("김철수", 25, "kim@example.com"))

# 에러 케이스
try:
    validate_user("", 25, "kim@example.com")
except ValidationError as e:
    print(f"검증 실패: {e}")

try:
    validate_user("김철수", 200, "kim@example.com")
except ValidationError as e:
    print(f"검증 실패: {e}")
```

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 예외 체이닝 (raise from) ⭐⭐⭐⭐

**문제**: 예외 체이닝을 사용하여 원인을 추적 가능한 에러 처리를 구현하세요.

**시나리오**:
설정 파일을 읽고 파싱하는 시스템을 만듭니다.

**요구사항**:
1. `ConfigError` 예외 클래스 정의
2. `load_config` 함수:
   - 파일 읽기 실패 → `ConfigError("설정 파일 로드 실패")` from 원래 에러
   - JSON 파싱 실패 → `ConfigError("설정 파일 형식 오류")` from 원래 에러
3. 체이닝된 에러의 원인(`__cause__`) 출력

**함수 시그니처**:
```python
import json

class ConfigError(Exception):
    """설정 관련 에러"""
    pass

def load_config(filepath):
    """
    JSON 설정 파일 로드
    
    Args:
        filepath: 설정 파일 경로
    
    Returns:
        dict: 설정 딕셔너리
    
    Raises:
        ConfigError: 설정 파일 문제 시 (원인 포함)
    """
    pass
```

**테스트**:
```python
# 테스트 파일 생성
with open('valid_config.json', 'w') as f:
    json.dump({'debug': True, 'port': 8080}, f)

with open('invalid_config.json', 'w') as f:
    f.write('{invalid json}')

# 정상 케이스
print(load_config('valid_config.json'))

# 에러 케이스 1: 파일 없음
try:
    load_config('nonexistent.json')
except ConfigError as e:
    print(f"에러: {e}")
    print(f"원인: {e.__cause__}")

# 에러 케이스 2: 잘못된 JSON
try:
    load_config('invalid_config.json')
except ConfigError as e:
    print(f"에러: {e}")
    print(f"원인: {e.__cause__}")
```

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 로깅 모듈 기초 ⭐⭐⭐

**문제**: `print` 대신 `logging` 모듈을 사용하여 데이터 처리 함수를 작성하세요.

**시나리오**:
여러 파일을 처리하면서 진행 상황과 에러를 로그로 남깁니다.

**요구사항**:
1. 로깅 설정:
   - 레벨: INFO
   - 포맷: `시간 - 레벨 - 메시지`
   - 파일: `process.log` (콘솔에도 출력)
2. `process_files` 함수:
   - 시작 시: INFO 로그
   - 각 파일 처리 시작: INFO 로그
   - 파일 없음: WARNING 로그 (계속 진행)
   - 기타 에러: ERROR 로그 (계속 진행)
   - 종료 시: INFO 로그 (성공/실패 개수)

**함수 시그니처**:
```python
import logging

def setup_logging():
    """로깅 설정"""
    pass

def process_file(filepath):
    """파일 처리 (시뮬레이션)"""
    # 파일을 읽고 줄 수를 반환한다고 가정
    with open(filepath) as f:
        return len(f.readlines())

def process_files(filepaths):
    """
    여러 파일 처리
    
    Args:
        filepaths: 파일 경로 리스트
    
    Returns:
        dict: {'성공': 성공_개수, '실패': 실패_개수}
    """
    pass
```

**테스트**:
```python
# 테스트 파일 생성
with open('test1.txt', 'w') as f:
    f.write('line1\nline2\nline3')

with open('test2.txt', 'w') as f:
    f.write('data')

# 실행
setup_logging()
files = ['test1.txt', 'test2.txt', 'nonexistent.txt']
result = process_files(files)
print(result)
```

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 실전: 데이터 파이프라인 에러 처리 ⭐⭐⭐⭐⭐

**문제**: 실무 수준의 데이터 파이프라인을 만드세요.

**시나리오**:
여러 소스에서 데이터를 수집하고 처리하는 파이프라인입니다.
- API 호출 (실패 가능)
- 데이터 검증 (형식 오류 가능)
- 파일 저장 (권한 오류 가능)

**요구사항**:
1. 커스텀 예외:
   - `DataPipelineError` (기본)
   - `DataFetchError` (데이터 수집 실패)
   - `DataValidationError` (검증 실패)
   - `DataSaveError` (저장 실패)

2. 재시도 로직:
   - API 호출 실패 시 최대 3회 재시도
   - 각 재시도 사이 1초 대기

3. 로깅:
   - 각 단계 진행 상황
   - 에러 발생 시 상세 정보
   - 최종 처리 결과 (성공/실패 통계)

4. 에러 체이닝:
   - 하위 에러를 상위 에러로 체이닝

**함수 시그니처**:
```python
import logging
import time
import random

# 커스텀 예외
class DataPipelineError(Exception):
    """데이터 파이프라인 기본 에러"""
    pass

class DataFetchError(DataPipelineError):
    """데이터 수집 에러"""
    pass

class DataValidationError(DataPipelineError):
    """데이터 검증 에러"""
    pass

class DataSaveError(DataPipelineError):
    """데이터 저장 에러"""
    pass

# 시뮬레이션 함수들
def fetch_data_from_api(source_id):
    """API에서 데이터 가져오기 (시뮬레이션)"""
    # 30% 확률로 실패
    if random.random() < 0.3:
        raise ConnectionError(f"API 연결 실패: {source_id}")
    
    return {
        'source_id': source_id,
        'value': random.randint(1, 100),
        'timestamp': time.time()
    }

def validate_data(data):
    """데이터 검증"""
    if 'value' not in data:
        raise ValueError("value 필드 없음")
    
    if not isinstance(data['value'], int):
        raise TypeError("value는 정수여야 함")
    
    if data['value'] < 0 or data['value'] > 100:
        raise ValueError("value는 0-100 범위여야 함")
    
    return True

def save_data(data, filepath):
    """데이터 저장"""
    import json
    
    with open(filepath, 'a') as f:
        json.dump(data, f)
        f.write('\n')

# 메인 함수
def fetch_with_retry(source_id, max_attempts=3):
    """
    재시도 로직이 있는 데이터 수집
    
    Args:
        source_id: 소스 ID
        max_attempts: 최대 시도 횟수
    
    Returns:
        dict: 수집된 데이터
    
    Raises:
        DataFetchError: 모든 시도 실패 시
    """
    pass

def process_source(source_id, output_file):
    """
    하나의 소스에서 데이터 수집 → 검증 → 저장
    
    Args:
        source_id: 소스 ID
        output_file: 출력 파일 경로
    
    Returns:
        bool: 성공 여부
    """
    pass

def run_pipeline(source_ids, output_file='pipeline_output.jsonl'):
    """
    데이터 파이프라인 실행
    
    Args:
        source_ids: 소스 ID 리스트
        output_file: 출력 파일 경로
    
    Returns:
        dict: {'성공': int, '실패': int, '실패_목록': list}
    """
    pass
```

**테스트**:
```python
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# 파이프라인 실행
sources = ['source_1', 'source_2', 'source_3', 'source_4', 'source_5']
result = run_pipeline(sources)

print("\n=== 파이프라인 결과 ===")
print(f"성공: {result['성공']}")
print(f"실패: {result['실패']}")
if result['실패_목록']:
    print(f"실패 소스: {result['실패_목록']}")
```

**보너스 과제**:
1. 실패한 소스만 재처리하는 `retry_failed` 함수 작성
2. 처리 시간을 측정하고 로그에 포함
3. 에러 통계를 딕셔너리로 반환 (에러 타입별 카운트)

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### 예외 처리 구조

| 키워드 | 실행 조건 | 용도 |
|--------|-----------|------|
| `try` | 항상 | 위험한 코드 실행 |
| `except` | 에러 발생 시 | 에러 처리 |
| `else` | 에러 없을 때 | 성공 시 실행 |
| `finally` | 항상 | 정리 작업 (리소스 해제) |

### 주요 예외 타입

```python
# 내장 예외
Exception              # 모든 예외의 기본 클래스
  ├─ ValueError        # 잘못된 값
  ├─ TypeError         # 잘못된 타입
  ├─ KeyError          # 딕셔너리 키 없음
  ├─ IndexError        # 인덱스 범위 초과
  ├─ FileNotFoundError # 파일 없음
  ├─ PermissionError   # 권한 없음
  ├─ TimeoutError      # 시간 초과
  └─ ConnectionError   # 연결 실패

# 사용자 정의 예외
class CustomError(Exception):
    pass
```

### 예외 처리 Best Practices

```python
# ❌ 나쁜 예: 모든 에러 무시
try:
    risky_operation()
except:
    pass

# ✅ 좋은 예: 구체적 에러 처리
try:
    risky_operation()
except ValueError as e:
    logging.error(f"값 오류: {e}")
except ConnectionError as e:
    logging.error(f"연결 오류: {e}")
    raise  # 재발생

# ❌ 나쁜 예: print로 에러 출력
except Exception as e:
    print(f"에러: {e}")

# ✅ 좋은 예: logging 사용
except Exception as e:
    logging.exception("처리 중 에러 발생")  # 스택 트레이스 포함
```

### 로깅 레벨

| 레벨 | 숫자 | 용도 | 예시 |
|------|------|------|------|
| DEBUG | 10 | 개발 디버깅 | 변수 값, 함수 호출 |
| INFO | 20 | 일반 정보 | 진행 상황 |
| WARNING | 30 | 경고 | 권장하지 않는 사용 |
| ERROR | 40 | 에러 | 예외 발생 |
| CRITICAL | 50 | 치명적 에러 | 시스템 다운 |

### 로깅 설정 패턴

```python
import logging

# 기본 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('app.log'),
        logging.StreamHandler()  # 콘솔 출력
    ]
)

# 사용
logger = logging.getLogger(__name__)
logger.info("시작")
logger.error("에러")
logger.exception("예외 발생")  # 스택 트레이스 포함
```

### 실무 패턴

**1. 재시도 로직**:
```python
def retry(func, max_attempts=3, delay=1):
    for attempt in range(max_attempts):
        try:
            return func()
        except Exception as e:
            if attempt == max_attempts - 1:
                raise
            logging.warning(f"재시도 {attempt+1}/{max_attempts}: {e}")
            time.sleep(delay)
```

**2. 컨텍스트 매니저**:
```python
# with 문 (권장)
with open('file.txt') as f:
    data = f.read()
# 자동으로 f.close() 호출

# try-finally (명시적)
f = open('file.txt')
try:
    data = f.read()
finally:
    f.close()
```

**3. 에러 체이닝**:
```python
try:
    low_level_operation()
except LowLevelError as e:
    raise HighLevelError("고수준 작업 실패") from e
    # __cause__ 속성에 원인 저장
```

### 💡 실무 활용 팁

1. **에러 처리 전략**:
   - 복구 가능: 로그 남기고 계속 진행
   - 복구 불가능: 에러 발생시키고 중단
   - 일시적 문제: 재시도 로직 적용

2. **로깅 vs Print**:
   - 개발 중: print 사용 가능
   - 배포 후: 반드시 logging 사용
   - 이유: 레벨 제어, 파일 저장, 포맷 통일

3. **예외 계층 구조**:
   - 프로젝트별 기본 예외 정의
   - 모듈별 구체적 예외 파생
   - 호출자가 적절한 레벨에서 처리

4. **성능 고려**:
   - 예외는 비용이 큼 (스택 트레이스 생성)
   - 정상 흐름 제어에 사용하지 말 것
   - EAFP vs LBYL 상황에 따라 선택

---

## 🎓 다음 단계

예외 처리와 로깅을 마스터했으니, Day3_2에서 **함수형 프로그래밍**을 배워봅시다!

- `map`, `filter`, `reduce`로 데이터 변환
- `lambda` 함수와 고차 함수
- `functools`와 `itertools` 활용

**다음 노트북**: `Day3_2_함수형_프로그래밍_Functional.ipynb`